## 💸 Stock Trend Analysis Framework

### 📥 Setups

Installation and Import of required packages

In [244]:
# !pip install -r requirements.txt

In [245]:
import requests

import yfinance as yf
import pandas as pd
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from dash import Dash, dcc, html, Input, Output, State
from dash.exceptions import PreventUpdate
from io import StringIO
import ast

Initialization of Analysis Parameters

In [246]:
TICKERS: list[str] = ["AAPL", "MSFT", "AMZN", "TSLA", "NVDA", "JNJ", "JPM", "XOM", "PG", "WMT", "AMT"]
MACROS: list[str] = ['Real GDP', 'FED Rates', 'CPI']
START_DATE: str = "2020-01-01"
END_DATE: str = "2025-07-18"

INDICATORS: list[str] = ['MA', 'RSI', 'MACD']

MA_PERIODS: list[int] = [50, 200]
VOL_PERIOD: int = 20
RSI_PERIOD: int = 14
MACD_WINDOWS: list[dict[str, str]] = [
    {'label': 'Classic (12, 26, 9)', 'value': (12, 26, 9)},
    {'label': 'Short-term (5, 35, 5)', 'value': (5, 35, 5)},
    {'label': 'Long-term (19, 39, 9)', 'value': (19, 39, 9)},
]

MA_CROSS_FRESHNESS: int = 4
MACD_NEUTRAL_THRESHOLD: float = 0.4

SHADE_COLOR: dict[str, str] = {
    'Strong Bullish': 'rgba(0, 200, 0, 0.6)',  # dark green
    'Bullish': 'rgba(0, 200, 0, 0.4)',  # light green
    'Strong Bearish': 'rgba(200, 0, 0, 0.6)',  # dark red
    'Bearish': 'rgba(200, 0, 0, 0.4)',  # light red
    'Neutral': 'rgba(85, 85, 85, 0.4)'  # light gray
}

Alpha_Vantage_API_TOKEN = "GBMV74GZ3G8DGXO2" # Replace with your own Alpha Vantage API token

### 🏗️ Data Acquisition

Function for Fetching Daily Price Data and Macros Data

In [247]:
def get_stock_data(tickers: list[str], start: str, end: str):
    """
    Fetches financial data from Yahoo Finance API for given tickers and date range.
    
    Parameters:
        tickers (list[str]): U.S. equity ticker symbols
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
        dict[str, pd.DataFrame]: Dictionary of DataFrames with date-indexed adjusted closing price data
    """
    
    data = {}

    for ticker in tickers:
        try:
            # Fetch data from Yahoo Finance and select relevant columns
            df = yf.download(ticker, start=start, end=end, auto_adjust=True)[['Close', 'Volume']]
            df.columns = ['Close', 'Volume']
            
            if not df.empty:
                data[ticker] = df
            else:
                print(f"No data found for {ticker} in the specified date range.")
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    return data

Function for Fetching GDP, CPI, and FED Rates

In [248]:
def get_macros():
    """
    Fetches Real GDP, CPI, and FED Rates from Alpha Vantage API

    Returns:
        dict[pd.DataFrame]: A dictionary of macros' DataFrames
    """

    request_map = {
        'Real GDP': {'rf': 'REAL_GDP', 'f': 'quarterly'}, 
        'FED Rates': {'rf': 'FEDERAL_FUNDS_RATE', 'f': 'daily'},
        'CPI': {'rf': 'CPI', 'f': 'monthly'}
    }

    result = {}

    for m in MACROS:
        request_url = f'https://www.alphavantage.co/query?function={request_map[m]['rf']}&interval={request_map[m]['f']}&apikey={Alpha_Vantage_API_TOKEN}'
        requestResponse = requests.get(request_url)

        # Check if the response is valid
        if requestResponse.status_code != 200:
            raise ValueError(f"Error fetching data.")

        # Extract Requested Data
        data = pd.DataFrame(requestResponse.json()['data'])

        result[m] = data

    return result

### 🧹 Data Cleaning and Preparation

Function for Filling Missing Values and Normalizing Values

In [249]:
def clean_data(data: dict[str, pd.DataFrame]):
    """
    Cleans and prepares the stock data for analysis.
    
    Parameters:
        data (dict[str, pd.DataFrame]): Dictionary of DataFrames indexed by ticker symbols

    Returns:
        dict[str, pd.DataFrame]: Dictionary of DataFrames with cleaned and normalized data
    """

    cleaned_data = {}

    for ticker, df in data.items():
        # Replace zeros in 'Close' with NaN
        df.loc[df['Close'] == 0, 'Close'] = np.nan

        # Forward and backward fill missing values
        df = df.ffill().bfill()
        
        # Normalize the 'Close' prices to a range of 0 to 1
        df['Close_Normalized'] = (df['Close'] - df['Close'].min()) / (df['Close'].max() - df['Close'].min())

        df = df[['Close', 'Close_Normalized', 'Volume']] # Change column order
        
        cleaned_data[ticker] = df

    return cleaned_data

Function for Clearning and Interpolating Macro Data

In [250]:
def clean_macros(data: dict[str, pd.DataFrame], start_date: str = START_DATE, end_date: str = END_DATE):
    """
    Cleans and prepares the macro data for analysis.
    
    Parameters:
        data (dict[str, pd.DataFrame]): Dictionary of DataFrames indexed by macros
        start_date (str): Start date in 'YYYY-MM-DD' format (Defaults to START_DATE)
        end_date (str): End date in 'YYYY-MM-DD' format (Defaults to END_DATE)

    Returns:
        dict[str, pd.DataFrame]: Dictionary of DataFrames with cleaned data
    """

    cleaned_data = {}

    for macro, df in data.items():
        # Convert 'date' column to datetime
        df["date"] = pd.to_datetime(df["date"])

        # Set 'date' as index
        df.set_index("date", inplace=True)
        
        # Ensure the column is sorted by date
        df = df.sort_index()

        # Convert the value column to numeric
        df = df.astype({'value': float})

        df = df.ffill().bfill()

        if macro == 'Real GDP':
            df = df.resample('D').asfreq().interpolate(method='polynomial', order=2)
        elif macro == 'CPI':
            df = df.resample('D').asfreq().interpolate(method='spline', order=1)

        df.rename(columns={'value': macro}, inplace=True)

        cleaned_data[macro] = df.loc[start_date:end_date]
    
    return cleaned_data

### 📊 Data Analysis

Functions for Calculating Return, Moving Average, and Volatility

In [251]:
def calculate_return(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the daily return of the stock based on the 'Close' price.

    Parameters:
        df (pd.DataFrame): DataFrame containing stock data with 'Close' prices

    Returns:
        pd.DataFrame: DataFrame with an additional 'Return' column
    """
    
    df['Return'] = df['Close'].pct_change()
    return df

def calculate_moving_average(df: pd.DataFrame, window: int) -> pd.DataFrame:
    """
    Calculates the moving average of the 'Close' price over a specified window.

    Parameters:
        df (pd.DataFrame): DataFrame containing stock data with 'Close' prices
        window (int): The number of periods over which to calculate the moving average
    
    Returns:
        pd.DataFrame: DataFrame with an additional column for the moving average
    """

    df[f'Moving_Avg_{window}'] = df['Close'].rolling(window=window).mean()
    return df

def calculate_volatility(df: pd.DataFrame, window: int) -> pd.DataFrame:
    """
    Calculates the volatility of the stock based on the 'Return' column over a specified window.

    Parameters:
        df (pd.DataFrame): DataFrame containing stock data with 'Return' column
        window (int): The number of periods over which to calculate the volatility
    
    Returns:
        pd.DataFrame: DataFrame with an additional column for the volatility
    """

    df['Rolling_Vol'] = df['Return'].rolling(window=window).std()
    return df

Function for Calculating all the Measures

In [252]:
def calculate_measures(data: dict[str, pd.DataFrame], ma_window: list[int] = MA_PERIODS, vol_window: int = VOL_PERIOD) -> dict:
    """
    Calculates the return, moving average and the volatility for each stock's closing price.
    
    Parameters:
        data (dict[str, pd.DataFrame]): Dictionary of DataFrames indexed by ticker symbols
        ma_window (list[int]): Window size(s) for moving average (Default to 50 and 200)
        vol_window (int): Window size for rolling volatility calculation (Default to 20)

    Returns:
        dict[str, dict[str, pd.DataFrame | float]]: Dictionary of DataFrames with volatility added
    """

    final_data = {}

    for ticker, df in data.items():
        df = calculate_return(df)
        for ma_w in ma_window:
            df = calculate_moving_average(df, ma_w)
        df = calculate_volatility(df, vol_window)

        final_data[ticker] = {
            'data': df,
            'overall_vol': float(df['Return'].std())
        }

    return final_data

Functions for Calculating RSI and MACD

In [253]:
def calculate_RSI(df: pd.DataFrame, window: int) -> pd.DataFrame:
    """
    Calculates the Relative Strength Index (RSI) for a given DataFrame.

    Parameters:
        df (pd.DataFrame): DataFrame containing stock data with 'Close' prices
        period (int): The number of periods to use for RSI calculation

    Returns:
        pd.DataFrame: DataFrame with an additional 'RSI' column
    """

    df = df.copy()

    avg_gain = (df['Return'].where(df['Return'] > 0, 0)).rolling(window=window).mean()
    avg_loss = (-df['Return'].where(df['Return'] < 0, 0)).rolling(window=window).mean()
    
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    return df

def calculate_MACD(df: pd.DataFrame, short_window: int, long_window: int, signal_window: int) -> pd.DataFrame:
    """
    Calculates the Moving Average Convergence Divergence (MACD) for a given DataFrame.

    Parameters:
        df (pd.DataFrame): DataFrame containing stock data with 'Close' prices
        short_window (int): Short-term EMA window
        long_window (int): Long-term EMA window
        signal_window (int): Signal line EMA window

    Returns:
        pd.DataFrame: DataFrame with additional EMA, MACD, MACD Signal, and MACD Histogram columns
    """

    df = df.copy()

    df['EMA_short'] = df['Close'].ewm(span=short_window, adjust=False).mean()
    df['EMA_long'] = df['Close'].ewm(span=long_window, adjust=False).mean()

    df['MACD'] = df['EMA_short'] - df['EMA_long']
    df['MACD_Signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
    df['MACD_Hist'] = df['MACD'] - df['MACD_Signal']
    
    return df

Function for Calculating Both Indicators

In [254]:
def calculate_indicators(data: dict[str, dict[str, pd.DataFrame | float]], 
                         rsi_window: int = RSI_PERIOD, short_window: int = MACD_WINDOWS[0]['value'][0], long_window: int = MACD_WINDOWS[0]['value'][1], signal_window: int = MACD_WINDOWS[0]['value'][2]):
    """
    Calculates the RSI and MACD for given DataFrames.

    Parameters:
        data (dict[str, dict[str, pd.DataFrame | float]]): Dictionary of DataFrames indexed by ticker symbols
        rsi_window (int): The number of periods to use for RSI calculation (Default to 14)
        short_window (int): Short-term EMA window (Default to 12)
        long_window (int): Long-term EMA window (Default to 26)
        signal_window (int): Signal line EMA window (Default to 9)

    Returns:
        dict[str, dict[str, pd.DataFrame | float]]: Dictionary of DataFrames with EMA, MACD, MACD Signal, and MACD Histogram added
    """

    final_data = {}

    for ticker, d in data.items():
        df = d['data']

        df = calculate_RSI(df, rsi_window)
        df = calculate_MACD(df, short_window, long_window, signal_window)

        final_data[ticker] = {
            'data': df,
            'overall_vol': d['overall_vol']
        }

    return final_data

Function for Calculating Macro Return 

In [255]:
def calculate_return_macros(data: dict[str, pd.DataFrame]) -> pd.DataFrame:
    """
    Calculates the daily return of the macro

    Parameters:
        data (dict[pd.DataFrame]): Dictionary of DataFrames containing macro data

    Returns:
        dict[pd.DataFrame]: Dictionary of DataFrames with an additional 'Return' column
    """
    
    for macro, df in data.items():
        df['Return'] = df[macro].pct_change()
    return data

Function for Finding Indicator Trends

In [272]:
def get_trend_conditions(df: pd.DataFrame, indicator: str) -> dict[str, pd.Series]:
    """
    Generate trend condition masks based on price and moving averages.

    Parameters
        df (pd.DataFrame): DataFrame containing at least the columns 'Close' and the moving averages specified in `ma_cols`.
        indicator (str): Indicator the trend bases on.

    Returns
        dict[str, pd.Series]: A dictionary mapping trend condition names to boolean Series indicating where each condition is met.
    """

    if indicator == 'MA':
        ma_cols = sorted([col for col in df.columns if 'Moving_Avg' in col], key=lambda x: int(x.split('_')[-1]))
        conditions = {
            'Neutral': ~(((df['Close'] >= df[ma_cols[0]]) & (df['Close'] >= df[ma_cols[1]])) |
                        ((df['Close'] <= df[ma_cols[0]]) & (df['Close'] <= df[ma_cols[1]]))),
            'Strong Bullish': (df['Close'] >= df[ma_cols[0]]) & (df['Close'] >= df[ma_cols[1]]) & \
                            (df[ma_cols[0]] >= df[ma_cols[1]]) & \
                            (df[ma_cols[0]].shift(MA_CROSS_FRESHNESS) < df[ma_cols[1]].shift(MA_CROSS_FRESHNESS)),
            'Bullish': (df['Close'] >= df[ma_cols[0]]) & (df['Close'] >= df[ma_cols[1]]),
            'Strong Bearish': (df['Close'] <= df[ma_cols[0]]) & (df['Close'] <= df[ma_cols[1]]) & \
                            (df[ma_cols[0]] < df[ma_cols[1]]) & \
                            (df[ma_cols[0]].shift(MA_CROSS_FRESHNESS) >= df[ma_cols[1]].shift(MA_CROSS_FRESHNESS)),
            'Bearish': (df['Close'] <= df[ma_cols[0]]) & (df['Close'] <= df[ma_cols[1]])
        }
    elif indicator == 'RSI':
        conditions = {
            'Bearish': (df['RSI'] > 70),  # Overbought
            'Bullish': (df['RSI'] < 30),  # Oversold
            'Neutral': (df['RSI'] <= 70) &(df['RSI'] >= 30)
        }
    else:  # MACD
        conditions = {
            'Neutral': (abs(df['MACD_Hist']) < MACD_NEUTRAL_THRESHOLD) & (abs(df['MACD'] < MACD_NEUTRAL_THRESHOLD)),
            'Strong Bullish': (df['MACD_Hist'] >= 0) & (df['MACD'] < 0),
            'Bullish': (df['MACD_Hist'] >= 0),
            'Strong Bearish': (df['MACD_Hist'] < 0) & (df['MACD'] > 0),
            'Bearish': (df['MACD_Hist'] < 0),
        }
    
    return conditions


def find_trends(data: dict[str, dict[str, pd.DataFrame | float]]):
    """
    Assign trend labels to each row of df based on conditions.

    Parameters:
        data (dict[str, dict[str, pd.DataFrame | float]]): Dictionary of DataFrames indexed by ticker symbols

    Returns:
        pd.DataFrame: DataFrame with trend labels
    """
    
    for ticker, d in data.items():
        df = d['data']
        
        for i in INDICATORS:
            df[f'Trend_{i}'] = 'Unclassified'

            priority_order = ['Neutral', 'Strong Bullish', 'Bullish', 'Strong Bearish', 'Bearish']

            conditions = get_trend_conditions(df, i)

            for label in priority_order:
                if label in conditions:
                    mask = conditions[label]

                    df.loc[mask & (df[f'Trend_{i}'] == 'Unclassified'), f'Trend_{i}'] = label

        df['Signal'] = 'Hold'
        df.loc[(((df['Trend_RSI'] == 'Bullish')) & ((df['Trend_MACD'] == 'Strong Bullish') | (df['Trend_MACD'] == 'Bullish'))), 'Signal'] = "Buy"
        df.loc[(((df['Trend_RSI'] == 'Bearish')) & ((df['Trend_MACD'] == 'Strong Bearish') | (df['Trend_MACD'] == 'Bearish'))), 'Signal'] = "Sell"

    return data


Function for Filtering Missing Values after all Calculations

In [257]:
def filter_data(data: dict[str, dict[str, pd.DataFrame | float]]):
    """
    Filters out rows of missing values for given stock dataFrames

    Parameters:
        data (dict[str, dict[str, pd.DataFrame | float]]): Dictionary of DataFrames indexed by ticker symbols

    Returns:
        dict[str, dict[str, pd.DataFrame | float]]: Dictionary of DataFrames without NaNs
    """

    final_data = {}

    for ticker, d in data.items():
        df = d['data']

        df = df.loc[df.dropna().index[0]:]

        final_data[ticker] = {
            'data': df,
            'overall_vol': d['overall_vol']
        }
    return final_data

def filter_macros(data: dict[str, pd.DataFrame]):
    """
    Filters out rows of missing values for given macro dataFrames

    Parameters:
        data (dict[str, pd.DataFrame]): Dictionary of DataFrames indexed by macros

    Returns:
        dict[str, pd.DataFrame]: Dictionary of DataFrames without NaNs
    """

    final_data = {}

    for macro, df in data.items():
        df = df.loc[df.dropna().index[0]:]

        final_data[macro] = df
    return final_data

Function for Preparing Data and Macro

In [258]:
def prepare_data(data: dict[str, pd.DataFrame], ma_window: list[int] = MA_PERIODS, vol_window: int = VOL_PERIOD, 
                 rsi_window: int = RSI_PERIOD, short_window: int = MACD_WINDOWS[0]['value'][0], long_window: int = MACD_WINDOWS[0]['value'][1], signal_window: int = MACD_WINDOWS[0]['value'][2]):
    """
    Prepares data by calculating measures, indicators, and trends

    Parameters:
        data (dict[str, pd.DataFrame]): Dictionary of DataFrames indexed by ticker symbols
        ma_window (list[int]): Window size(s) for moving average (Default to 50 and 200)
        vol_window (int): Window size for rolling volatility calculation (Default to 20)
        rsi_window (int): The number of periods to use for RSI calculation (Default to 14)
        short_window (int): Short-term EMA window (Default to 12)
        long_window (int): Long-term EMA window (Default to 26)
        signal_window (int): Signal line EMA window (Default to 9)
    
    Returns:
        dict[str, dict[str, pd.DataFrame | float]]: Dictionary of DataFrames without NaNs
    """

    data = calculate_measures(data, ma_window, vol_window)
    data = calculate_indicators(data, rsi_window, short_window, long_window, signal_window)
    data = find_trends(data)
    data = filter_data(data)

    return data

In [273]:
t = clean_data(get_stock_data(TICKERS, START_DATE, END_DATE))
t = prepare_data(t)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


### 📈 Visualization

Function for Shading according to a Scheme

In [261]:
def shade_by_scheme(df: pd.DataFrame, indicator: str, fig: go.Figure, row: int, col: int, colors: dict[str, str] = SHADE_COLOR):
    """
    Shades regions on a Plotly figure based on boolean masks defined in the scheme.

    Parameters:
        df (pd.DataFrame): Dataframe to shade
        indicator (str): Indicator to shade based on
        fig (go.Figure): A Plotly figure object where the shading rectangles will be added
        row (int): The row number in the subplot grid where shading should be applied
        col (int): The column number in the subplot grid where shading should be applied
        colors (dict[str, str]): A dictionary mapping labels to color strings used for shading (Defaults to SHADE_COLOR)
    """

    if f'Trend_{indicator}' not in df.columns:
        raise ValueError("Column not found in DataFrame.")

    for trend, color in colors.items():
        mask = (df[f'Trend_{indicator}'] == trend)

        in_region = False
        start = None

        for i in range(len(df)):
            if mask.iloc[i] and not in_region:
                start = df.index[i]
                in_region = True
            elif not mask.iloc[i] and in_region:
                end = df.index[i]
                fig.add_vrect(x0=start, x1=end, fillcolor=color,
                              line_width=0, row=row, col=col)
                in_region = False

        if in_region:  # If region continues till the end
            fig.add_vrect(x0=start, x1=df.index[-1], fillcolor=color,
                          line_width=0, row=row, col=col)

Function for Marking Signals

In [266]:
def mark_signals(df: pd.DataFrame, fig: go.Figure, row: int):
    """
    Marks buy/sell signals on a Plotly figure.

    Parameters:
        df (pd.DataFrame): Dataframe to mark signals for
        fig (go.Figure): A Plotly figure object where the marks will be added
        row (int): The row number of the subplot to add traces to.
    """

    # Buy signals - green upward arrows
    buy_signals = df[df['Signal'] == 'Buy']
    fig.add_trace(go.Scatter(
        x=buy_signals.index,
        y=buy_signals['Close'],
        mode='markers',
        marker=dict(symbol='arrow-up', color='green', size=12),
        name='Buy Signal'
    ), row=row, col=1)

    # Sell signals - red downward arrows
    sell_signals = df[df['Signal'] == 'Sell']
    fig.add_trace(go.Scatter(
        x=sell_signals.index,
        y=sell_signals['Close'],
        mode='markers',
        marker=dict(symbol='arrow-down', color='red', size=12),
        name='Sell Signal'
    ), row=row, col=1)

Functions for Plotting Prices, Moving Averages, Volume, RSI, and MACD

In [263]:
def plot_price_and_moving_average(df: pd.DataFrame, fig: go.Figure, row: int) -> None:
    """
    Adds traces and shades for closing price and all moving averages to a specific subplot row.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'Close' and moving average columns.
        fig (go.Figure): Plotly Figure object to which traces will be added.
        row (int): The row number of the subplot to add traces to.
    """

    fig.add_trace(
        go.Scatter(x=df.index, y=df['Close'], name='Close', line=dict(color='black')),
        row=row, col=1
    )
    for c in df.columns:
        if 'Moving_Avg' in c:
            fig.add_trace(go.Scatter(x=df.index, y=df[c], name=c, line=dict(dash='longdashdot')), row=row, col=1)
    
    shade_by_scheme(df, 'MA', fig, row, 1)

def plot_volume(df: pd.DataFrame, fig: go.Figure, row: int) -> None:
    """
    Adds a bar chart for volume data to a specific subplot row.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'Volume' column.
        fig (go.Figure): Plotly Figure object to which the volume bar chart will be added.
        row (int): The row number of the subplot to add the volume chart to.
    """

    fig.add_trace(
        go.Bar(x=df.index, y=df['Volume'], name='Volume', marker_color='gray'),
        row=row, col=1
    )

def plot_rsi(df: pd.DataFrame, fig: go.Figure, row: int) -> None:
    """
    Adds a line chart for RSI with threshold lines and shades to a specific subplot row.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'RSI' column.
        fig (go.Figure): Plotly Figure object to which RSI traces will be added.
        row (int): The row number of the subplot to add the RSI plot to.
    """
    
    fig.add_trace(
        go.Scatter(x=df.index, y=df['RSI'], name='RSI', line=dict(color='gold')),
        row=row, col=1
    )
    fig.add_hline(y=70, line_dash='dash', line_color='red', row=row, col=1)
    fig.add_hline(y=30, line_dash='dash', line_color='blue', row=row, col=1)
    
    shade_by_scheme(df, 'RSI', fig, row, 1)

def plot_macd(df: pd.DataFrame, fig: go.Figure, row: int) -> None:
    """
    Adds line charts for MACD and MACD Signal, and a bar chart for MACD Histogram to a specific subplot row.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'MACD', 'MACD_Signal', and 'MACD_Hist' columns.
        fig (go.Figure): Plotly Figure object to which MACD traces will be added.
        row (int): The row number of the subplot to add MACD-related plots to.
    """

    fig.add_trace(
        go.Scatter(x=df.index, y=df['MACD'], name='MACD', line=dict(color='blue')),
        row=row, col=1
    )
    fig.add_trace(
        go.Scatter(x=df.index, y=df['MACD_Signal'], name='MACD Signal', line=dict(color='orange')),
        row=row, col=1
    )
    fig.add_trace(
        go.Bar(x=df.index, y=df['MACD_Hist'], name='MACD Hist', marker_color='purple'),
        row=row, col=1
    )

    shade_by_scheme(df, 'MACD', fig, row, 1)

Function for Plotting all the Technicals

In [264]:
def plot_technicals(df: pd.DataFrame, ticker: str) -> None:
    """
    Creates a 4-row technical analysis dashboard with subplots for price & moving averages, volume, RSI, and MACD.

    Parameters:
        df (pd.DataFrame): DataFrame containing all necessary columns for plotting technical indicators.
        ticker (str): Stock ticker symbol used for the plot title.
    """

    fig = make_subplots(
        rows=4, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03,
        row_heights=[0.4, 0.2, 0.2, 0.2]
    )

    plot_price_and_moving_average(df, fig, row=1)
    plot_volume(df, fig, row=2)
    plot_rsi(df, fig, row=3)
    plot_macd(df, fig, row=4)
    mark_signals(df, fig, row=1)

    fig.update_layout(
        height=1000,
        title_text=f"{ticker} Technical Analysis Dashboard",
        title_x=0.5,
        showlegend=True
    )
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.update_yaxes(title_text="RSI", row=3, col=1)
    fig.update_yaxes(title_text="MACD", row=4, col=1)
    fig.update_xaxes(title_text="Date", row=4, col=1)

    return fig

In [274]:
plot_technicals(t['AAPL']['data'], 'AAPL')

Functions for Plotting Volatility vs Time, Return, Volume, RSI, and MACD

In [ ]:
def plot_volatility(df: pd.DataFrame, overall_volatility: int, fig: go.Figure, row: int, col: int) -> None:
    """
    Adds traces for rolling volatility over time to a specific subplot row and column.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'Close' and moving average columns.
        overall_volatility (int): Volatility of the entire DataFrame
        fig (go.Figure): Plotly Figure object to which traces will be added.
        row (int): The row number of the subplot to add traces to.
        col (int): The column number of the subplot to add to.
    """

    fig.add_trace(
        go.Scatter(x=df.index, y=df['Rolling_Vol'], name='Rolling Volatility', line=dict(color='blue')), 
        row=row, col=col
    )
    fig.add_trace(
        go.Scatter(x=df.index, y=[overall_volatility] * len(df), name='Overall Volatility', line=dict(color='blue', dash='dash')), 
        row=row, col=col
    )

def scatter_volatility(df: pd.DataFrame, y_axis: str, fig: go.Figure, row: int, col: int) -> None:
    """
    Adds a scatterplot for volatility assessment to a specific subplot row and column.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'Volume' column.
        y_axis (str): Column to scatter for y-axis
        fig (go.Figure): Plotly Figure object to which the volume bar chart will be added.
        row (int): The row number of the subplot to add the volume chart to.
        col (int): The column number fo the subplot to add to.
    """

    fig.add_trace(
        go.Scatter(x=df['Rolling_Vol'], y=df[y_axis], mode='markers', name=y_axis),
        row=row, col=col
    )


Function for Plotting all the Volatility Measures

In [ ]:
def scatterplot_volatility(df: pd.DataFrame, overall_volatility: int, ticker: str):
    """
    Creates a 3-row volatility assessment dashboard with subplots for Volatility vs Time, Return, Volume, RSI, and MACD

    Parameters:
        df (pd.DataFrame): DataFrame containing all necessary columns for plotting technical indicators.
        overall_volatility (int): Volatility of the entire DataFrame
        ticker (str): Stock ticker symbol used for the plot title.
    """

    fig = make_subplots(
        rows=3, cols=2,
        specs=[[{"colspan": 2}, None],
            [{}, {}],
            [{}, {}]],                  
        row_heights=[0.4, 0.3, 0.3],
        vertical_spacing=0.1
    )

    plot_volatility(df, overall_volatility, fig, 1, 1)
    scatter_volatility(df, 'MACD', fig, 2, 1)    
    scatter_volatility(df, 'RSI', fig, 2, 2)   
    scatter_volatility(df, 'Volume', fig, 3, 1)   
    scatter_volatility(df, 'Return', fig, 3, 2)   

    fig.update_layout(
        height=1000,
        title_text=f"{ticker} Volatility Assessment Dashboard",
        title_x=0.5,
        showlegend=True
    )
    fig.update_yaxes(title_text="Rolling Volatility", row=1)
    fig.update_xaxes(title_text="Date", row=1)
    fig.update_yaxes(title_text="MACD", row=2, col=1)
    fig.update_yaxes(title_text="RSI", row=2, col=2)
    fig.update_xaxes(title_text="Rolling Volatility", row=2)
    fig.update_yaxes(title_text="Volume", row=3, col=1)
    fig.update_yaxes(title_text="Return", row=3, col=2)
    fig.update_xaxes(title_text="Rolling Volatility", row=3)

    return fig

Function for Plotting Correlation of Macros with Stock Technicals

In [ ]:
def plot_correlations(df_macro: pd.DataFrame, df_stock: pd.DataFrame, macro: str, ticker: str):
    """
    Returns a heatmap showing the correlation matrix of selected financial indicators from macroeconomic and stock data.

    Parameters:
        df_macro (pd.DataFrame): DataFrame containing macroeconomic variables.
        df_stock (pd.DataFrame): DataFrame containing stock-related variables.
        macro (str): Macro of plotting.
        ticker (str): Ticker symbol of plotting.

    Returns:
        plotly.graph_objects.Figure: A Plotly Figure object containing the correlation heatmap.
    """

    df_combined = pd.concat([df_macro.add_suffix(f'_{macro}'), df_stock.add_suffix(f'_{ticker}')], axis = 1)

    correlations = df_combined[[f'Return_{macro}', f'Return_{ticker}', f'Rolling_Vol_{ticker}', f'RSI_{ticker}', f'MACD_{ticker}']].corr()
    correlations = correlations[[f'Return_{macro}']].drop(index=f'Return_{macro}')

    fig = px.imshow(
        correlations,
        text_auto=".2f",
        color_continuous_scale='RdBu',
        zmin=-1,
        zmax=1,
        aspect='auto',
    )

    fig.update_layout(title=dict(text=f"{macro} Correlations Heatmap", x=0.5, xanchor='center'), margin=dict(l=40, r=40, t=60, b=40))
    
    return fig

Functions for Creating Dashboard

In [ ]:
def create_layout(tickers: list[str]):
    """
    Constructs and returns the full layout of the Dash stock dashboard.
    The layout includes:
        - A title header
        - Dropdown selectors for ticker and date range
        - Sliders and dropdowns for technical indicator parameters
        - Two `dcc.Store` components to hold raw and processed data
        - An interval component to trigger data loading once
        - Two graphs for price with technical overlays and volatility

    Parameters:
        tickers (List[str]): List of available stock ticker symbols to populate the dropdown.

    Returns:
        html.Div: The complete layout as a Dash HTML Div component.
    """

    return html.Div([
        html.H1(
            "Stock Dashboard",
            style={'textAlign': 'center', 'fontFamily': 'Arial, sans-serif', 'margin-bottom': '20px'}
        ),
        html.Div([
            html.Div([
                html.Label("Ticker", style={'fontFamily': 'Arial, sans-serif', 'marginBottom': '5px'}),
                dcc.Dropdown(
                    id='ticker-dropdown',
                    options=[{'label': t, 'value': t} for t in tickers],
                    value=tickers[0],
                    style={'width': '300px', 'fontFamily': 'Arial, sans-serif'}
                ),
            ], style={'display': 'flex',
                      'flexDirection': 'column',
                      'alignItems': 'flex-start'
            }),
            html.Div([
                html.Label("Data Range", style={'fontFamily': 'Arial, sans-serif', 'marginBottom': '5px'}),
                dcc.DatePickerRange(
                    id='date-range-picker',
                    display_format='YYYY-MM-DD',
                    style={'width': '300px', 'fontFamily': 'Arial, sans-serif'}
                )
            ], style={'display': 'flex',
                      'flexDirection': 'column',
                      'alignItems': 'flex-start'
            }),
        ], style={'display': 'flex',
                 'justifyContent': 'center',
                 'alignItems': 'flex-start',
                 'gap': '40px',
                 'marginBottom': '30px'
        }),
        html.Div([
            html.Label("Moving Average Window 1", style={'fontFamily': 'Arial, sans-serif'}),
            dcc.Slider(id='moving_average_1_window', min=2, max=250, step=1, value=MA_PERIODS[0], 
                        marks={i: str(i) for i in range(5, 251, 5)},tooltip={'placement': 'bottom', 'always_visible': True})
        ]),
        html.Div([
            html.Label("Moving Average Window 2", style={'fontFamily': 'Arial, sans-serif'}),
            dcc.Slider(id='moving_average_2_window', min=2, max=250, step=1, value=MA_PERIODS[1], 
                        marks={i: str(i) for i in range(5, 251, 5)},tooltip={'placement': 'bottom', 'always_visible': True})
        ]),
        html.Div([
            html.Label("RSI Window", style={'fontFamily': 'Arial, sans-serif'}),
            dcc.Slider(id='rsi_window', min=5, max=30, step=1, value=RSI_PERIOD, 
                        marks={i: str(i) for i in range(5, 31, 5)},tooltip={'placement': 'bottom', 'always_visible': True})
        ]),
        html.Div([
            html.Div([
                html.Label("Volatility Window", style={'fontFamily': 'Arial, sans-serif'}),
                dcc.Dropdown(id='volatility-window', value=VOL_PERIOD,
                         options=[{'label': str(x), 'value': x} for x in [5, 10, 14, 20, 30, 40, 50, 60]], clearable=False,
                         style={'width': '200px', 'fontFamily': 'Arial, sans-serif'})
            ]),
            html.Div([
                html.Label("MACD Windows", style={'fontFamily': 'Arial, sans-serif'}),
                dcc.Dropdown(id='macd-windows-dropdown', value=str(MACD_WINDOWS[0]['value']),
                         options=[{'label': macd_w['label'], 'value': str(macd_w['value'])} for macd_w in MACD_WINDOWS], clearable=False,
                         style={'width': '200px', 'fontFamily': 'Arial, sans-serif'})
            ])
        ], style={'display': 'flex',
                 'justifyContent': 'left',
                 'alignItems': 'flex-start',
                 'gap': '40px',
                 'marginBottom': '20px'
        }),
        dcc.Store(id='macro-data-store'),
        dcc.Store(id='raw-data-store'),
        dcc.Store(id='data-store'),
        dcc.Interval(id='interval-once', interval=1*1000, n_intervals=0, max_intervals=1),
        html.Div([
            dcc.Graph(id='price-technicals-graph'),
            dcc.Graph(id='volatility-graph')
        ]),
        html.Div([
            dcc.Graph(id='gdp-corr-grpah', style={'flex': '1'}),
            dcc.Graph(id='fed-corr-grpah', style={'flex': '1'}),
            dcc.Graph(id='cpi-corr-grpah', style={'flex': '1'}),
        ], style={
            'display': 'flex',
            'justifyContent': 'space-between',
            'gap': '20px',
            'marginTop': '10px'
        })
    ], style={
        'backgroundColor': '#ffffff',
        'minHeight': '100vh'
    })

def register_callbacks(app: Dash):
    """
    Registers the Dash callbacks for updating graphs.

    Parameters:
        app (Dash): The Dash app instance.
    """

    @app.callback(
        Output('raw-data-store', 'data'),
        Output('macro-data-store', 'data'),
        Input('interval-once', 'n_intervals'),
        prevent_initial_call=True
    )
    def load_all_data(_: int):
        """
        Callback to load and preprocess raw stock data once at app load.

        Parameters:
        - _ (int): Dummy input from `interval-once` used to trigger this callback once.

        Returns:
        - str: JSON-encoded dictionary where each macro maps to its cleaned DataFrame (serialized).
        - str: JSON-encoded dictionary where each ticker maps to its cleaned DataFrame (serialized).
        """

        raw_data = clean_data(get_stock_data(TICKERS, START_DATE, END_DATE))
        
        raw_data_dict = {
            ticker: df.to_json(date_format='iso', orient='split')
            for ticker, df in raw_data.items()
        }

        macro_data = filter_macros(calculate_return_macros(clean_macros(get_macros())))

        macro_data_dict = {
            macro: df.to_json(date_format='iso', orient='split')
            for macro, df in macro_data.items()
        }
        
        return raw_data_dict, macro_data_dict

    @app.callback(
        Output('data-store', 'data'),
        Output('date-range-picker', 'min_date_allowed'),
        Output('date-range-picker', 'max_date_allowed'),
        Output('date-range-picker', 'start_date'),
        Output('date-range-picker', 'end_date'),
        Input('raw-data-store', 'data'),
        Input('moving_average_1_window', 'value'),
        Input('moving_average_2_window', 'value'),
        Input('rsi_window', 'value'),
        Input('volatility-window', 'value'),
        Input('macd-windows-dropdown', 'value'),
        State('date-range-picker', 'start_date'),
        State('date-range-picker', 'end_date'),
        prevent_initial_call=True
    )
    def update_data(raw_data_dict: str, ma1: int, ma2: int, rsi: int, vol: int, macd_str: str, curr_start: str, curr_end: str):
        """
        Update the processed data and the allowed/selected date range in the DatePicker.

        Parameters:
        - data_json (str): JSON-encoded raw stock data.
        - ma1 (int): Window for first moving average.
        - ma2 (int): Window for second moving average.
        - rsi (int): RSI window.
        - vol (int): Volatility window.
        - macd_str (str): Comma-separated MACD window parameters (e.g., "12,26,9").
        - curr_start (str): Currently selected start date.
        - curr_end (str): Currently selected end date.

        Returns:
        - Tuple:
            - Dict[str, Dict[str, str]]: Processed and filtered stock data per ticker.
            - pd.Timestamp: Minimum available date in the data.
            - pd.Timestamp: Maximum available date in the data.
            - str: Updated start_date for DatePicker.
            - str: Updated end_date for DatePicker.
        """

        if not raw_data_dict:
            raise PreventUpdate

        # Load raw data back into DataFrame
        all_data = {
            ticker: pd.read_json(StringIO(df_json), orient='split')
            for ticker, df_json in raw_data_dict.items()
        }

        macd = ast.literal_eval(macd_str) # Parse MACD values from dropdown string

        # Compute & clean
        all_data = calculate_measures(all_data, [ma1, ma2], vol)
        all_data = calculate_indicators(all_data, rsi, *macd)
        all_data = filter_data(all_data)

        # Output structure: {ticker: {"data": df_json}}
        output_data = {
            ticker: {"data": ticker_data['data'].to_json(date_format='iso', orient='split'),
                     'overall_vol': ticker_data['overall_vol']
                     }
            for ticker, ticker_data in all_data.items()
        }

        all_dates = pd.concat([df['data'].index.to_series() for df in all_data.values()])
        min_date = all_dates.min().date()
        max_date = all_dates.max().date()

        # Decide whether to update start/end dates
        new_start = str(min_date) if curr_start is None or (curr_start < str(min_date) or curr_start > str(max_date)) else curr_start
        new_end = str(max_date) if curr_end is None or (curr_end < str(min_date) or curr_end > str(max_date)) else curr_end

        return output_data, min_date, max_date, new_start, new_end

    @app.callback(
        Output('price-technicals-graph', 'figure'),
        Output('volatility-graph', 'figure'),
        Output('gdp-corr-grpah', 'figure'),
        Output('fed-corr-grpah', 'figure'),
        Output('cpi-corr-grpah', 'figure'),
        Input('macro-data-store', 'data'),
        Input('data-store', 'data'),
        Input('ticker-dropdown', 'value'),
        Input('date-range-picker', 'start_date'),
        Input('date-range-picker', 'end_date'),
        prevent_initial_call=True
    )
    def update_graphs(macro_data: dict[str, str],stock_data: dict[str, dict[str, str]], ticker: str, start: str, end: str):
        """
        Update the price/technical graph and volatility graph based on selected ticker and date range.

        Parameters:
        - macro_data (dict[str, str]): JSON-encoded processed data per macro.
        - stock_data (dict[str, dict[str, str]]): JSON-encoded processed data per ticker.
        - ticker (str): Selected stock ticker.
        - start (str): Selected start date.
        - end (str): Selected end date.

        Returns:
        - Tuple:
            - fig_price (plotly Figure): Price and technical indicators plot.
            - fig_volatility (plotly Figure): Volatility scatter plot.
        """

        if not macro_data or not stock_data or ticker not in stock_data:
            raise PreventUpdate
        
        gdp_df = pd.read_json(StringIO(macro_data[MACROS[0]]), orient='split').loc[start:end]
        fed_df = pd.read_json(StringIO(macro_data[MACROS[1]]), orient='split').loc[start:end]
        cpi_df = pd.read_json(StringIO(macro_data[MACROS[2]]), orient='split').loc[start:end]

        stock_df = pd.read_json(StringIO(stock_data[ticker]['data']), orient='split').loc[start:end]

        fig_price = plot_technicals(stock_df, ticker)
        fig_volatility = scatterplot_volatility(stock_df, stock_data[ticker]['overall_vol'], ticker)
        fig_gdp_corr = plot_correlations(gdp_df, stock_df, 'Real GDP', ticker)
        fig_fed_corr = plot_correlations(fed_df, stock_df, 'FFR', ticker)
        fig_cpi_corr = plot_correlations(cpi_df, stock_df, 'CPI', ticker)

        return fig_price, fig_volatility, fig_gdp_corr, fig_fed_corr, fig_cpi_corr

In [ ]:
app = Dash(__name__)

app.layout = create_layout(TICKERS)
register_callbacks(app)

if __name__ == '__main__':
    app.run(debug=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
